In [1]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import dill as pickle
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import const
from romar import utils
from romar import backend as bkd
from romar.systems import BoxAd
from romar.roms import CoBRAS

import romar.postproc.plotting as pltt

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/ROMAr/romar/examples/database"
path_to_bases = "/home/zanardi/Codes/ML/ROMAr/run/rad_on_test1/max_mom_2/cobras_bases.p"
path_to_mask = "/home/zanardi/Codes/ML/ROMAr/run/rad_on_test1/max_mom_2/rom_mask.txt"
path_to_data = "/home/zanardi/Codes/ML/ROMAr/run/rad_on_test1/data/"
cases = ["lrho_lT", "lrho_hT", "hrho_lT", "hrho_hT", "mrho_mT"]

In [5]:
system = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  rad_dtb=path_to_dtb + "/rates/rad_fit.p",
  use_rad=True,
  use_proj=False,
  use_tables=False
)
system.compute_c_mat(max_mom=2, state_specs=False, include_temp=False)

Plot bases

In [6]:
bases = pickle.load(open(path_to_bases, "rb"))

In [ ]:
for i in range(10):
  nb = str(i+1)
  b = bases["phi"][:,i]
  pltt.plot_dist_2d(
    x=np.arange(len(b)),
    y=b,
    labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % ("phi", nb)],
    scales=["linear", "linear"],
    markersize=1,
    # figname=path + f"/{name}_{nb.zfill(2)}",
    save=False,
    show=True
  )

In [ ]:
for i in range(10):
  nb = str(i+1)
  b = bases["psi"][:,i]
  pltt.plot_dist_2d(
    x=np.arange(len(b)),
    y=b,
    labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % ("psi", nb)],
    scales=["linear", "linear"],
    markersize=1,
    # figname=path + f"/{name}_{nb.zfill(2)}",
    save=False,
    show=True
  )

Load test cases

In [9]:
testcases = {k: utils.load_case(filename=path_to_data + "case_" + k + ".p") for k in cases}

In [ ]:
for k, testcase in testcases.items():
  # Testcase
  print(k)
  print("="*20)
  t, y, rho = [testcase[k] for k in ("t", "y", "rho")]
  system.mix.set_rho(rho)
  n, Th, Te = system.get_prim(y)
  # Species
  for s in system.mix.species.values():
    print(s.name)
    for i in s.indices:
      plt.loglog(t, n[i])
    plt.show()
    plt.close()
  # Temperatures
  print("Th-Te")
  plt.semilogx(t, Th)
  plt.semilogx(t, Te)
  plt.show()
  plt.close()

Test ROM

In [17]:
rdim = 8

system.set_rom(
  phi=bases["phi"][:,:rdim],
  psi=bases["psi"][:,:rdim],
  mask=path_to_mask
)

In [ ]:
for k, testcase in testcases.items():
  # Testcase
  print(k)
  print("="*20)
  # FOM solution
  t, y0, y, rho = [testcase[k] for k in ("t", "y0", "y", "rho")]
  system.mix.set_rho(rho)
  prim_fom = system.get_prim(y)
  mom_fom = system.compute_mom(prim_fom[0])
  # ROM solution
  yrom, _ = system.solve_rom(t, y0, rho)
  prim_rom = system.get_prim(yrom)
  mom_rom = system.compute_mom(prim_rom[0])
  nt = len(yrom.T)
  nt
  # Temperatures
  print("Th-Te")
  for i in range(2):
    plt.semilogx(t[:nt], prim_fom[1+i][:nt])
    plt.semilogx(t[:nt], prim_rom[1+i][:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
  plt.show()
  plt.close()
  # Species
  for s in system.mix.species.values():
    print(s.name)
    for i in s.indices:
      plt.loglog(t[:nt], prim_fom[0][i,:nt])
      plt.loglog(t[:nt], prim_rom[0][i,:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
    plt.show()
    plt.close()
  # Moments
  for s in system.mix.species.values():
    if s.name != "em":
      for m in range(2):
        print(s.name+f" - Moment: {m}")
        plt.semilogx(t[:nt], mom_fom[s.name][f"m{m}"][:nt])
        plt.semilogx(t[:nt], mom_rom[s.name][f"m{m}"][:nt], ls="--", color=plt.gca().lines[-1].get_color(), lw=2.0)
        plt.yscale("log" if m == 0 else "linear")
        plt.show()
        plt.close()